In [5]:
# https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html
# https://scikit-learn.org/stable/modules/outlier_detection.html#outlier-detection

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from pprint import pprint
from scipy import signal

In [7]:
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
pylab.rcParams.update(params)

## Preprocessing

In [8]:
basePath = '../../zju-gaitacc/'
LENGTH = 1000

In [9]:
def session_to_dataframe(session):
    df = pd.DataFrame(columns=range(3 * LENGTH + 1))
    
    i = 0
    for subj in Path(basePath + session).glob('subj_*'):
        for rec in subj.glob('*'):
            with open(rec.joinpath('3.txt')) as f:
                # extract the relevant range
                with open(rec.joinpath('useful.txt')) as use:
                    begin_rec, end_rec = use.readline().split(',')

                begin_rec = int(begin_rec)
                end_rec = int(end_rec)

                # read the content of the file
                lines = list(map(lambda line: [float(x) for x in line.strip().split(',')], f.readlines()))

                # select only relevant portion of the recording and resample it to fixed length
                lines = list(map(lambda x: signal.resample(x[begin_rec:end_rec], LENGTH), lines))

                # select label of measurement
                label = int(subj.name.split('_')[1])

                flat_list = [el for l in lines for el in l]
                flat_list.append(label)
                df.loc[i] = flat_list
                i += 1
                
    return df

In [10]:
df.to_csv('full_rec.csv', sep='\t', encoding='utf-8')

NameError: name 'df' is not defined

In [11]:
df_s0 = session_to_dataframe('session_0')
df_s0.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000


In [12]:
df_s0.shape

(0, 3001)

In [ ]:
df_s1 = session_to_dataframe('session_1')
df_s1.head()

In [ ]:
df_s1.shape

In [ ]:
df_s1.to_csv('full_session1.csv', sep='\t', encoding='utf-8')

## Data loading

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_s1 = pd.read_csv('full_session1.csv', sep='\t')

In [ ]:
df_s1.shape

In [ ]:
df_s1.head()

In [ ]:
select_classes = range(1, 21)
# df = pd.concat([df_s1_128, df_s2_128])
df_s1 = df_s1.loc[df_s1[df_s1.columns[-1]].isin(select_classes)]
y = df_s1[df_s1.columns[-1]].values
df_s1.drop([df_s1.columns[-1]], axis=1, inplace=True)
X = df_s1.values

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
df_s0 = pd.read_csv('full_session0.csv', sep='\t')

In [ ]:
y = df_s0[df_s0.columns[-1]].values
df_s0.drop([df_s0.columns[-1]], axis=1, inplace=True)
X = df_s0.values

In [ ]:
s0_X_train, ss0_X_test, s0_y_train, s0_y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, init='pca', random_state=0)
X_2d = tsne.fit_transform(X)

In [ ]:
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import StrMethodFormatter
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 12),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
pylab.rcParams.update(params)

In [ ]:
data = np.concatenate((X_2d, y.reshape(y.shape[0], 1)), axis=1)
viz_df = pd.DataFrame(data=data, columns=['x', 'y', 'class'])

In [ ]:
import seaborn as sns
fg = sns.FacetGrid(data=viz_df, hue='class', hue_order=np.unique(y), aspect=1.5, height=10)
fg.map(plt.scatter, 'x', 'y').add_legend()

In [ ]:
target_ids = np.unique(y)
colors = cm.rainbow(np.linspace(0, 1, len(target_ids)))
for i, c in zip(target_ids, colors):
    plt.scatter(X_2d[y == i, 0], X_2d[y == i, 1], label=i, color=c)
plt.legend()
plt.show()

## One-class SVM

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.OneClassSVM(nu=0.2, kernel="rbf", gamma='auto')
clf.fit(X_train)

In [ ]:
train_pred = clf.predict(X_train)

In [ ]:
str(np.count_nonzero(train_pred == -1)) + '/' + str(len(X_train))

In [ ]:
test_pred = clf.predict(X_test)

In [ ]:
str(np.count_nonzero(test_pred == -1)) + '/' + str(len(X_test))

In [ ]:
s0_pred = clf.predict(X)

In [ ]:
str(np.count_nonzero(s0_pred == -1)) + '/' + str(len(X))

## USPS

In [ ]:
import h5py

In [ ]:
with h5py.File('../data/usps.h5', 'r') as hf:
    train = hf.get('train')
    X_train = train.get('data')[:]
    y_train = train.get('target')[:]
    test = hf.get('test')
    X_test = test.get('data')[:]
    y_test = test.get('target')[:]
    
X_train.shape, y_train.shape

In [ ]:
X = np.concatenate((X_train, X_test), axis=0)

In [ ]:
y = np.concatenate((y_train, y_test), axis=0)

In [ ]:
clf = svm.OneClassSVM(nu=0.2, kernel="rbf", gamma='auto')
clf.fit(X_train)